In [20]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [15]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_df = pd.read_csv("lending_data.csv")

# Review the DataFrame
lending_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [16]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_df['loan_status']

# Separate the X variable, the features
X = lending_df.drop(columns=['loan_status'])

In [17]:
# Review the y variable Series
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [18]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [19]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [21]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [22]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs',max_iter = 200, random_state=1)

# Fit the model using training data
lr_model = classifier.fit(X_train, y_train)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [23]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
36831,0,0
75818,0,1
36563,0,0
13237,0,0
43292,0,0
...,...,...
38069,0,0
36892,0,0
5035,0,0
40821,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [24]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, predictions)

0.9442676901753825

In [27]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,18679,80
Actual 1,67,558


In [28]:
# Print the classification report for the model
print("Classification Report")
target_names = ["healthy loan","high-risk loan"]
print(classification_report(y_test, predictions, target_names = target_names))

Classification Report
                precision    recall  f1-score   support

  healthy loan       1.00      1.00      1.00     18759
high-risk loan       0.87      0.89      0.88       625

      accuracy                           0.99     19384
     macro avg       0.94      0.94      0.94     19384
  weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:**
•	**Precision** Measuring our model’s accuracy told us overall how many times we were correct when predicting the status. of the loans. For the "healthy loan" class, the precision is 1.00, indicating that the model predicts this label with perfect precision. For the "high-risk loan" class, the precision is 0.87, meaning that 87% of the predicted high-risk loans are correct.

•	**Recall**, also known as sensitivity or true positive rate tells you how many times the model was able to detect a specific  label. The recall for the "healthy loan" class is 1.00, indicating that the model captures all the actual healthy loans. The recall for the "high-risk loan" class is 0.89, meaning that the model identifies 89% of the high-risk loans correctly.

•	The **F1-score** is the mean of precision and recall and provides a balanced measure of the model's performance. For the "healthy loan" class, the F1-score is 1.00, indicating perfect performance. The F1-score for the "high-risk loan" class is 0.88, representing a good balance between precision and recall for this label.

•	The **accuracy score** is 99%, 99 correct predictions out of 100 predictions made.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points.

In [29]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
model_ros = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = model_ros.fit_resample(X_train, y_train)

In [32]:
# Count the distinct values of the resampled labels data
display(y_resampled.value_counts())

display(X_resampled.value_counts())

0    56277
1    56277
Name: loan_status, dtype: int64

loan_size  interest_rate  borrower_income  debt_to_income  num_of_accounts  derogatory_marks  total_debt
19900.0    11.596         89700            0.665552        13               2                 59700         185
19100.0    11.246         86400            0.652778        12               2                 56400         163
19300.0    11.316         87100            0.655568        12               2                 57100         162
18500.0    10.976         83900            0.642431        12               2                 53900         162
19300.0    11.310         87000            0.655172        12               2                 57000         156
                                                                                                           ... 
17800.0    10.696         81300            0.630996        11               2                 51300           1
12300.0    8.371          59400            0.494949        6                1                 29400           1

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [34]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier_ros = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
classifier_ros.fit(X_resampled, y_resampled)

# Make a prediction using the testing data
predictions_ros = classifier_ros.predict(X_test)
pd.DataFrame({"Prediction": predictions_ros, "Actual": y_test})

,Prediction,Actual
36831,0,0
75818,1,1
36563,0,0
13237,0,0
43292,0,0
...,...,...
38069,0,0
36892,0,0
5035,0,0
40821,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [35]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, predictions_ros)

0.9959744975744975

In [36]:
# Generate a confusion matrix for the model
matrix = confusion_matrix(y_test, predictions_ros)
matrix_df = pd.DataFrame(
   matrix, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
display(matrix_df)

,Predicted 0,Predicted 1
Actual 0,18668,91
Actual 1,2,623


In [37]:
# Print the classification report for the model
print("ROS Classification Report")
target_names = ["healthy loan","high-risk loan"]
print(classification_report(y_test, predictions_ros, target_names = target_names))

ROS Classification Report
                precision    recall  f1-score   support

  healthy loan       1.00      1.00      1.00     18759
high-risk loan       0.87      1.00      0.93       625

      accuracy                           1.00     19384
     macro avg       0.94      1.00      0.96     19384
  weighted avg       1.00      1.00      1.00     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:**
•**Precision**. For the "healthy loan" labels, the precision is 1.00, indicating that the model predicts this label with perfect precision. For the "high-risk loan" label, the precision is 0.87, meaning that 87% of the predicted high-risk loans are correct.

•	The recall for both the "healthy loan" and "high-risk loan" labels is 1.00, indicating that the model captures all the actual instances for both labels.

•**F1-Score**.For the "healthy loan" class, the F1-score is 1.00, indicating excellent performance. The F1-score for the "high-risk loan" class is 0.93, representing a good balance between precision and recall for this label.

•	The accuracy score is 100%, thus our model performs exceptionally well.

After resampling, the second model performs better. We do see the number of false negatives significantly decline from 67 to 2. The false positives are  more by 11 in the second model than the first. Our most undesirable outcome for these models is have someone be predicted negative when they are actually a high-risk defaulter. The second model limits that.
